# Sample Select for FDA Dataset
Run this only once the first time, otherwise start at Test Run

In [2]:
# from datasets import Dataset
import pandas as pd
drugs_at_fda = pd.read_excel('../../data/openfda-04232024.xlsx').reset_index(drop=True).drop('Unnamed: 0', axis=1)
drugs_at_fda
search_space = drugs_at_fda[drugs_at_fda['clinical_studies'].isnull()==False].reset_index(drop=True)
search_space = search_space[search_space['brand_name'].isnull()==False].reset_index(drop=True)

search_space[0:5]

KeyboardInterrupt: 

In [7]:
print(len(search_space))

18946


### RANDOM SELECTION
Run this block only once, load the yaml otherwise

In [ ]:
import random
import yaml

# Generate 500 unique random integers between 0 and 999
# Make sure the range is large enough to accommodate the sample size
# random_indices = random.sample(range(len(search_space)), 3800)

# Saving the list to a YAML file
# with open('20240424_random_indices_trial.yaml', 'w') as file:
#     yaml.dump(random_indices, file)

# print("500 unique random indices saved to unique_random_indices.yaml.")


with open('20240424_random_indices_trial.yaml','r') as f:
    random_indices = f.readlines()

def clean_index(line):
    return line.split(' ')[1].split('\n')[0]

sample = []
for line in random_indices:
    sample.append(clean_index(line))

search_space = search_space.iloc[sample].reset_index(drop=True)


sample[0:5]


NameError: name 'search_space' is not defined

In [12]:
search_space
search_space.to_excel('20240424_trial_searchspace.xlsx')

## Test Run

In [1]:
import pandas as pd

search_space = pd.read_excel('20240424_trial_searchspace.xlsx').reset_index(drop=True).drop('Unnamed: 0', axis=1)
search_space[0:5]


,brand_name,application_number,adverse_reactions,clinical_studies,indications_and_usage,contraindications,warnings_and_cautions,warnings,precautions,pharmacokinetics,purpose,clinical_pharmacology,active_ingredient,stop_use,boxed_warning,pharmacodynamics,pharmacogenomics
0,Dsuvia,NDA209128,6 ADVERSE REACTIONS The following serious adve...,14 CLINICAL STUDIES The efficacy and safety of...,1 INDICATIONS AND USAGE DSUVIA is indicated fo...,4 CONTRAINDICATIONS Use of DSUVIA is contraind...,5 WARNINGS AND PRECAUTIONS Opioid-Induced Hype...,NaN,NaN,12.3 Pharmacokinetics Absorption A single subl...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,WARNING: SERIOUS AND LIFE-THREATENING RISKS FR...,12.2 Pharmacodynamics Effects on the Central N...,NaN
1,ROPINIROLE HYDROCHLORIDE,ANDA090429,6 ADVERSE REACTIONS The following adverse reac...,14 CLINICAL STUDIES 14.1 Parkinson's Disease T...,1 INDICATIONS AND USAGE Ropinirole Tablets are...,4 CONTRAINDICATIONS Ropinirole tablets are con...,5 WARNINGS AND PRECAUTIONS · Sudden onset of s...,NaN,NaN,12.3 Pharmacokinetics Ropinirole displayed lin...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,12.2 Pharmacodynamics Clinical experience with...,NaN
2,Amlodipine Besylate,ANDA076846,6 ADVERSE REACTIONS Most common adverse reacti...,14 CLINICAL STUDIES 14.1 Effects in Hypertensi...,1 INDICATIONS AND USAGE Amlodipine besylate ta...,4 CONTRAINDICATIONS Amlodipine besylate tablet...,5 WARNINGS AND PRECAUTIONS Symptomatic hypoten...,NaN,NaN,NaN,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,12.2 Pharmacodynamics Hemodynamics: Following ...,NaN
3,Rabeprazole sodium,ANDA202376,6 ADVERSE REACTIONS Most common adverse reacti...,14 CLINICAL STUDIES 14.1 Healing of Erosive or...,1 INDICATIONS AND USAGE Rabeprazole sodium del...,4 CONTRAINDICATIONS Patients with a history of...,5 WARNINGS AND PRECAUTIONS Gastric Malignancy ...,NaN,NaN,12.3 Pharmacokinetics After oral administratio...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,12.2 Pharmacodynamics Antisecretory Activity T...,NaN
4,Phentermine hydrochloride,ANDA205017,6 ADVERSE REACTIONS The following adverse reac...,14 CLINICAL STUDIES In relatively short-term c...,1 INDICATIONS AND USAGE Phentermine hydrochlor...,4 CONTRAINDICATIONS • History of cardiovascula...,5 WARNINGS AND PRECAUTIONS Coadministration wi...,NaN,NaN,12.3 Pharmacokinetics Following the administra...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,12.2 Pharmacodynamics Typical actions of amphe...,NaN


In [13]:
import statistics

lengths = []
for text in search_space['clinical_studies']:
    # print(len(text))
    lengths.append(len(text))
print(statistics.stdev(lengths))
statistics.mean(lengths)


6741.559783129706


8121.756578947368

In [2]:
len(search_space)

3800

In [30]:
import pandas as pd
import llm
from tqdm.auto import tqdm

# Load the model once outside the loop
model = llm.get_model('mistral-7b-instruct-v0')

# Simulated start point and DataFrame creation
start_point = 2000

test_run = search_space[start_point:start_point+1000].copy()
test_run['outcome_raw'] = None

# Ensure the DataFrame column exists to avoid KeyErrors
if 'clinical_studies' not in test_run or 'brand_name' not in test_run:
    raise ValueError("Required columns are missing in the DataFrame.")

position = start_point
for idx, row in tqdm(test_run.iterrows(), total=test_run.shape[0]):
    # if position == start_point + 16:
    #     position += 1
    #     continue
    text_to_process = f'Read the following clinical studies for {row["brand_name"]}. Identify the primary treatment outcomes for each patient group. Format your answer using this JSON format: {{"result": {{"outcome": {{"metric": "string", "value": "string", "descriptors": "string"}}}}}} \n Text: {row["clinical_studies"][0:2000]}'
    response = model.prompt(text_to_process)
    test_run.at[idx, 'outcome_raw'] = response.text()
    position += 1


100%|██████████| 1000/1000 [4:18:22<00:00, 15.50s/it] 


In [34]:
test_run.to_excel('20240424_trial_results2000-3000.xlsx') #on 2000 now

In [33]:
test_run

,brand_name,application_number,adverse_reactions,clinical_studies,indications_and_usage,contraindications,warnings_and_cautions,warnings,precautions,pharmacokinetics,purpose,clinical_pharmacology,active_ingredient,stop_use,boxed_warning,pharmacodynamics,pharmacogenomics,outcome_raw
2000,Benazepril hydrochloride,ANDA076267,6 ADVERSE REACTIONS Because clinical trials ar...,14 CLINICAL STUDIES Hypertension Adult Patient...,1 INDICATIONS AND USAGE Benazepril hydrochlori...,4 CONTRAINDICATIONS Benazepril hydrochloride i...,5 WARNINGS AND PRECAUTIONS Angioedema: Discont...,NaN,NaN,NaN,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,WARNING: FETAL TOXICITY When pregnancy is dete...,NaN,NaN,"{""result"": {\n""outcome"": {\n""metric"": ""blood ..."
2001,Metoprolol Tartrate,ANDA077739,6 ADVERSE REACTIONS The following adverse reac...,14 CLINICAL STUDIES 14.1 Hypertension In contr...,1 INDICATIONS AND USAGE Metoprolol tartrate ta...,4 CONTRAINDICATIONS Metoprolol tartrate tablet...,5 WARNINGS AND PRECAUTIONS Abrupt cessation ma...,NaN,NaN,12.3 Pharmacokinetics Absorption : The estimat...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,12.2 Pharmacodynamics Clinical pharmacology st...,NaN,"{""result"": {\n""outcome"": {\n""metric"": ""Treatm..."
2002,KETOROLAC TROMETHAMINE,ANDA074802,ADVERSE REACTIONS Adverse reaction rates incre...,CLINICAL STUDIES dult Patients In a postoperat...,INDICATIONS & USAGE Carefully consider the pot...,CONTRAINDICATIONS (see also BOXED WARNING ) Ke...,NaN,WARNINGS (See also BOXED WARNING .) The total ...,PRECAUTIONS General Ketorolac tromethamine can...,NaN,NaN,CLINICAL PHARMACOLOGY Pharmacodynamics Ketorol...,NaN,NaN,BOXED WARNING BOXED WARNING,NaN,NaN,"{\n""result"": {\n""outcome"": {\n""metric"": ""Morp..."
2003,Bupropion Hydrochloride SR,ANDA079095,6 ADVERSE REACTIONS The following adverse reac...,14 CLINICAL STUDIES The efficacy of the immedi...,1 INDICATIONS AND USAGE Bupropion hydrochlorid...,4 CONTRAINDICATIONS • Bupropion hydrochloride ...,5 WARNINGS AND PRECAUTIONS • Seizure risk: The...,NaN,NaN,12.3 Pharmacokinetics Bupropion is a racemic m...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,WARNING: SUICIDAL THOUGHTS AND BEHAVIORS; AND ...,NaN,NaN,"{""result"": {\n""outcome"": {\n""metric"": ""Hamilt..."
2004,Betamethasone Dipropionate,ANDA076592,6 ADVERSE REACTIONS The most common adverse re...,14 CLINICAL STUDIES The safety and efficacy of...,1 INDICATIONS AND USAGE Betamethasone Dipropio...,4 CONTRAINDICATIONS Betamethasone Dipropionate...,5 WARNINGS AND PRECAUTIONS Effects on endocrin...,NaN,NaN,12.3 Pharmacokinetics No pharmacokinetics tria...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,12.2 Pharmacodynamics Vasoconstrictor Assay Tr...,NaN,"{\n""result"": {\n""outcome"": {\n""metric"": ""Effi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,APRISO,NDA022301,6 ADVERSE REACTIONS • The most common adverse ...,14 CLINICAL STUDIES 14.1 Ulcerative Colitis Tw...,1 INDICATIONS AND USAGE APRISO capsules are in...,4 CONTRAINDICATIONS APRISO is contraindicated ...,5 WARNINGS AND PRECAUTIONS • Renal impairment ...,NaN,NaN,12.3 Pharmacokinetics Absorption The pharmacok...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN,"{""result"": {\n""outcome"": {\n""metric"": ""relaps..."
2996,Azithromycin Dihydrate,ANDA208250,6 ADVERSE REACTIONS Most common adverse reacti...,14 CLINICAL STUDIES 14.1 Adult Patients Acute ...,1 INDICATIONS AND USAGE Azithromycin is a macr...,4 CONTRAINDICATIONS Patients with known hypers...,5 WARNINGS AND PRECAUTIONS Serious (including ...,NaN,NaN,12.3 Pharmacokinetics Following oral administr...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,12.2 Pharmacodynamics Based on animal models o...,NaN,"{""result"": {\n""outcome"": {\n""metric"": ""Clinic..."
2997,ciprofloxacin,ANDA076639,6 ADVERSE REACTIONS The following serious and ...,14 CLINICA

In [32]:
test_run['outcome_raw'][2159]

' {"result": {\n"outcome": {\n"metric": "Complete Clearance of Actinic Keratosis Lesions",\n"value": "30 Days Post-Treatment",\n"descriptors": "Diclofenac Sodium Gel Vehicle p-value Study 1 <0.001, Diclofenac Sodium Gel Vehicle p-value Study 2 0.061, Diclofenac Sodium Gel Vehicle p-value Study 3 0.221"\n}\n}}'

Consider gpt4all-13b-snoozy-q4_0.gguf or wizardlm-13b-v1.2.Q4_0.gguf for a larger more powerful model

In [7]:
### OLD FUNCTION LOOP ###
# start_point = 0

# test_run = search_space[start_point:start_point+50]
# test_run['outcome_raw'] = "None"
# import pdb
# import llm
# import ast
# from tqdm import tqdm
# model = llm.get_model('mistral-7b-instruct-v0')

# position = start_point
# for clincal_study in tqdm(test_run['clinical_studies']):

#     if position == start_point + 16:
#         position += 1
#         pass
#     else:

#         response = model.prompt(f'Read the follow clinical studies for {test_run["brand_name"][position]}. Identify the primary treatment outcomes for each patient group. Format your answer using this JSON format: {{"result": {{"outcome": {{"metric": "string","value": "string","descriptors": "string"}}}}}} \n Text: {test_run["clinical_studies"][position]}')

#         test_run['outcome_raw'][position] = response.text()
#         position += 1

/var/folders/5t/sfw5tjx56m10xb861_pd3wfm0000gq/T/ipykernel_5767/3811091761.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_run['outcome_raw'] = "None"
  0%|          | 0/50 [00:00<?, ?it/s]

: 

In [3]:
test_run.to_excel('20240325_trial_1-10.xlsx',sheet_name='20240325')